Есть датфрейм с authors, text, annotation и title. Текст предобработан, стоп-слова удалены. У одной статьи может быть несколько авторов, тогда они записаны в одной ячейке authors и разделены запятой. 

Функции:
- для вычисления всех соавторов одного человека по ФИО, они могут быть записаны в разных ячейках датафрейма. 

- функция для векторизации текста.  

- функция для вычисления коэффициента схожести.

- функция для рекомендации новых соавторов. 

В функцию для рекомендации новых соавторов на вход ФИО автора. На выходе ФИО автора, названия всех статей в которых он участвовал, список его соавторов, список трёх рекомендуемых новых авторов/коллективов авторов, названия их статей и коэффициент близости. Нельзя рекомендовать автора самому себе и рекомендовать ему его текущих соавторов. Необходимо предусмотреть ситуацию когда рекомендуется несколько человек, написавших одну и ту же статью, при совпадении коэффициента близости у новых соавторов необходимо объединять их ФИО через запятую, писать их общую статью один раз и на 2 и 3 месте писать новых рекомендуемых авторов. При совпадении 2 и 3 рекомендации также следует их объединять и на третье место выводить следующих по коэффициенту схожести коллектив авторов/автора. Вывод должен быть с использованием pprint.

In [262]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np

In [248]:
df = pd.read_excel('date_it_lemm.xlsx')

In [158]:
df['authors'] = df['authors'].str.split(',') #авторов через запятую разделяем

Функция для поиска всех соавторов:

In [320]:
def get_coauthors(df, author_name):
    coauthors = set()
    for authors_list in df['authors'].str.split(', '):
        if author_name in authors_list:
            coauthors.update(authors_list)
    coauthors.discard(author_name)  # удаляем самого автора
    return list(coauthors)

get_coauthors(df, 'Тест А.Б.')

['Тест Г.Ф.', 'Тест З.Ы.', 'Тест В.Ф.']

Векторизация текста:

In [326]:
def vectorize_texts(texts):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    return tfidf_matrix, vectorizer

Схожесть между целевыми статьями и всеми остальными

In [328]:
def calculate_similarity(tfidf_matrix, target_indices, all_indices):
    similarities = cosine_similarity(tfidf_matrix[target_indices], tfidf_matrix[all_indices])
    return similarities.max(axis=0)

Основная функция с рекомендацией

In [386]:
def recommend_new_coauthors(df, author_name):
    
    target_mask = df['authors'].str.contains(author_name, regex=False) #данные автора
    target_indices = df[target_mask].index.tolist()
    
    if not target_indices:
        return {"error": "Автор не найден"}
    
    current_coauthors = get_coauthors(df, author_name) #текущие соавторы
    
    tfidf_matrix, _ = vectorize_texts(df['text']) #векторизация
    
    all_indices = df.index.tolist() 
    similarities = calculate_similarity(tfidf_matrix, target_indices, all_indices) #схожести

    results = pd.DataFrame({
        'index': all_indices,
        'similarity': similarities,
        'authors': df['authors'],
        'title': df['title']
    })
    
    results = results[
        (results['similarity'] < 1.0) &  #исключаем статьи автора
        (~results['authors'].isin([author_name]))  #исключаем соавторов
    ]
    
    #группировка по коэф схожести
    grouped = results.groupby('similarity').agg({
        'authors': lambda x: ', '.join(sorted(set(', '.join(x).split(', ')))),
        'title': lambda x: ', '.join(sorted(set(x)))
    }).reset_index().sort_values('similarity', ascending=False)
    
    recommendations = []
    used_authors = set(current_coauthors + [author_name]) #формируем рекомендации
    
    for _, row in grouped.iterrows():
        candidates = [a.strip() for a in row['authors'].split(',') if a.strip() not in used_authors]
        if candidates:
            unique_candidates = []
            for candidate in candidates:
                if candidate not in used_authors:
                    unique_candidates.append(candidate)
                    used_authors.add(candidate)
            
            if unique_candidates:
                recommendations.append({
                    'authors': ', '.join(unique_candidates),
                    'titles': row['title'],
                    'similarity': round(row['similarity'], 2)
                })
        
        if len(recommendations) >= 3:
            break
    
    #итоговый вывод
    output = {
        "Автор": author_name,
        "Текущие соавторы": current_coauthors,
        "Статьи": df[target_mask]['title'].tolist(),
        "Рекомендуемые соавторы": recommendations[:3]
    }
    
    return output

In [388]:
rec_list = recommend_new_coauthors(df, 'Большаков А. О.')
pprint.pprint(rec_list)

{'Автор': 'Большаков А. О.',
 'Рекомендуемые соавторы': [{'authors': 'Воеводин Вл В., Жолудев Ю. А., '
                                        'Соболев С. И., Стефанов К. С.',
                             'similarity': 0.44,
                             'titles': 'Эволюция системы метакомпьютинга '
                                       'X-Com'},
                            {'authors': 'Здитовец А. Л.',
                             'similarity': 0.43,
                             'titles': 'ОСНОВНЫЕ ТЕХНОЛОГИИ И ФРЕЙМВОРКИ ДЛЯ '
                                       'БЕКЕНД-РАЗРАБОТКИ НА JAVA'},
                            {'authors': 'Аньчков Максим Геннадьевич, Иванов '
                                        'Сергей Владимирович',
                             'similarity': 0.4,
                             'titles': 'Разработка и реализация алгоритмов '
                                       'передачи данных в телекоммуникационных '
                                       'системах'

In [360]:
#супер красивый вывод от нейросети, оставлю для доработки
def print_recommendations(result):
    print("=" * 70)
    print(f"РЕКОМЕНДАЦИИ ДЛЯ АВТОРА: {result['author']}".center(70))
    print("=" * 70)
    
    print("\n📚 Статьи автора:")
    for i, article in enumerate(result['articles'], 1):
        print(f"   {i}. {article}")
    
    print("\n🤝 Текущие соавторы:")
    print("\n".join(f"   - {coauth}" for coauth in result['current_coauthors']) or "   Нет соавторов")
    
    print("\n🔥 Рекомендуемые новые соавторы:")
    for i, rec in enumerate(result['recommendations'], 1):
  #      print(rec)
        authors = "".join(rec['author'])
        articles = "\n      ".join(rec['articles'])
        print(f"\n{i}. {authors}")
        print(f"   📑 Статьи: {articles}")
        print(f"   🎯 Схожесть: {rec['score']:.4f}")
    print("=" * 70)

In [362]:
result = get_author_recommendations(df, "Большаков А. О.")
print_recommendations(result)

               РЕКОМЕНДАЦИИ ДЛЯ АВТОРА: Большаков А. О.               

📚 Статьи автора:
   1. АВТОМАТИЗАЦИЯ ПРОЦЕССОВ МОНИТОРИНГА И ИНВЕНТАРИЗАЦИИ ИНФОРМАЦИОННО-ТЕХНОЛОГИЧЕСКОЙ ИНФРАСТРУКТУРЫ, ПРИМЕНЯЕМОЙ В УЧЕБНОМ ПРОЦЕССЕ

🤝 Текущие соавторы:
   - Леонтьев А. С.

🔥 Рекомендуемые новые соавторы:

1. Воеводин Вл В.
   📑 Статьи: Эволюция системы метакомпьютинга X-Com
   🎯 Схожесть: 0.4426

2. Здитовец А. Л.
   📑 Статьи: ОСНОВНЫЕ ТЕХНОЛОГИИ И ФРЕЙМВОРКИ ДЛЯ БЕКЕНД-РАЗРАБОТКИ НА JAVA
   🎯 Схожесть: 0.4311

3. Иванов Сергей Владимирович
   📑 Статьи: Разработка и реализация алгоритмов передачи данных в телекоммуникационных системах
   🎯 Схожесть: 0.4032
